# Demonstration RAG-Eval


Quick test: see if things work as planned

In [1]:
from rag_eval.protocol import LLMClient
from rag_eval.client import AzureClient
from rag_eval.components import ClaimExtractor, EntailmentJudge
from rag_eval.data_models import EvalContainer, ClaimVerdict
from rag_eval.evaluators import AnswerPrecisionEvaluator, RetrievalPrecisionEvaluator
from rag_eval.coordinator import RAGEvaluator
from rag_eval.helpers import _safe_json
import json
from data.rag_eval_samples import test_examples
from openai import AzureOpenAI
from dotenv import load_dotenv
import os



In [2]:

api_key = os.getenv("OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
base_url = os.getenv("OPENAI_CHATCOMPLETIONS_URL")
model = os.getenv("OPENAI_DEPLOYMENT")
password = os.getenv("CORRECT_PASSWORD")

llm = AzureClient(
    api_key=api_key,
    api_version=api_version,
    base_url=base_url,
    model=model
)


In [3]:
coord = RAGEvaluator(llm_extractor=llm, llm_judge=llm)
data = test_examples[0:2]


for i, ex in enumerate(data, start=1):
    print(f"EXAMPLE {i}")
    results = coord.evaluate_all(ex)
    print(json.dumps(results, indent=2))

EXAMPLE 1
{
  "answer": {
    "gt_claims": [
      "Italy's capital is Rome.",
      "Rome has about 2.8 million inhabitants in the city."
    ],
    "gen_claims": [
      "Italy's capital is Rome.",
      "Rome has 4 million inhabitants."
    ],
    "gt_claim_recall": 0.5,
    "hallucination_precision": 0.5,
    "gt_claim_verdicts": [
      {
        "claim": "Italy's capital is Rome.",
        "verdict": "supported",
        "rationale": "the evidence explicitly states that italy's capital is rome, which directly supports the claim. the additional information about the population does not contradict the claim about the capital city.",
        "doc_indices": null
      },
      {
        "claim": "Rome has about 2.8 million inhabitants in the city.",
        "verdict": "contradicted",
        "rationale": "the claim states that rome has about 2.8 million inhabitants, while the evidence states that rome has 4 million inhabitants. the numbers are not approximately equal, so the claim is

# Test Agent


In [11]:
# force reload of agent-script after changes 
import importlib, inspect, agent.main as main

#print("Module file:", main.__file__)      
main = importlib.reload(main)               # force reload the module
#print("Reloaded file:", main.__file__)

# show the exact source Python is executing
#print(inspect.getsource(main.food_agent))

In [12]:
# load agent and pipeline for indexing docs
import agent.main as main
from rag.indexing import pdf_files, index_pipeline

In [6]:
index_pipeline.run({"converter": {"sources": pdf_files}})


incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
Batches: 100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


{'writer': {'documents_written': 51}}

In [15]:
main.food_agent("hi")

Goodbye 🍕


In [5]:
print(pdf_files)

[WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/Antons_Pizza_Doggish_Delights_Menu_reportlab.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/Chez_Anton_Menu_Français_Classic_reportlab.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/cocktail_info_voll.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/frankreich_info_voll.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/franzoesisch_kueche_endversion.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/Il_Sogno_de_Anton_Menu_Doggish_reportlab.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/italienisch_kueche_endversion.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/italien_info_voll.pdf'), WindowsPath('C:/Users/ANDERBE/AI-Agent-POC-git/AI-Agent/data/RAG_docs/mexikanisch_kueche_endversion.pdf'), WindowsPath('C:/Users/ANDERB